In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# set user agent & headers for crawling
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) " + \
            "Chrome/37.0.2062.94 Safari/537.36"
headers = {"User-Agent": user_agent}

In [3]:
def total_page():
    url = url = "http://nsdi.go.kr/?menuno=2775"
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    result = divmod( int(dom.select_one(".allTxt span").text.replace(",","")), 10 )
    page = result[0]
    if result[1] > 0:
        page += 1
    return page

# function test code
total_page()

482

In [4]:
def page_crawling(page, df):
    url = "http://nsdi.go.kr/?menuno=2775&pageIndex=" + str(page)
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select(".bl_list tr")[2:]
    
    result_list = []
    
    for element in elements:
        local = element.select("td")[1].get("onclick").split("'")[1]
        regno = element.select("td")[1].get("onclick").split("'")[3]
        company = element.select("td")[1].text
        url = "http://nsdi.go.kr/?menuno=2775&devAct=view&sido_cd=" + local + "&ed_regno=" + regno
        dic = {"company":company, "local":local, "regno":regno, "url":url}
        df.loc[len(df)] = dic
    return df

# function test code
company_df = pd.DataFrame(columns=["company", "local", "regno", "url"])
company_df = page_crawling(10, company_df)
company_df.head(3)

,company,local,regno,url
0,(주)건일토건,48,경남130008,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
1,(주)건축사사무소 길인,47,경북100008,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
2,(주)건축사사무소신건축개발,11,서울100027,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...


In [5]:
def company_crawling(sido_cd, ed_regno, df):
    url = "http://nsdi.go.kr/?menuno=2775&devAct=view&sido_cd=" + str(sido_cd) + "&ed_regno=" + ed_regno + "&type=3#cont_area"
    response = requests.get(url, headers=headers)
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select(".business_table")
    
    for element in elements:
        year = element.select("tr")[1].select("td")[0].text
        business = element.select("tr")[2].select("td")[0].text.replace(",","")
        benefit = element.select("tr")[2].select("td")[2].text.replace(",","")
        land_width = element.select("tr")[2].select("td")[4].text
        build_width = element.select("tr")[2].select("td")[6].text
        nsdi_type = element.select("tr")[4].select("td")[0].text
        company_name = element.select("tr")[4].select("td")[1].text
        date1 = element.select("tr")[4].select("td")[2].text
        date2 = element.select("tr")[4].select("td")[3].text
        date3 = element.select("tr")[4].select("td")[4].text 
        dic = {
            "regno":ed_regno, 
            "year":year, 
            "business":business, 
            "benefit":benefit, 
            "land_width":land_width, 
            "build_width":build_width,
            "nsdi_type" : nsdi_type,
            "company_name":company_name,
            "date1":date1,
            "date2":date2,
            "date3":date3,
        }
        df.loc[len(df)] = dic
    return df

# function test code
detail_df = pd.DataFrame(
    columns=[
        "regno", "year", "business", "benefit", "land_width", "build_width", 
        "nsdi_type", "company_name", "date1", "date2", "date3"
    ]
)
for idx, row in company_df.iterrows():
    local = row["local"]
    regno = row["regno"]
    detail_df = company_crawling(local, regno, detail_df)
    
detail_df

,regno,year,business,benefit,land_width,build_width,nsdi_type,company_name,date1,date2,date3
0,경남130008,2014,1531,809,"1,816",790,C,(주)건일토건,2013.07.03,2014.05.12,2014.06.14
1,경남130008,2013,1800,0,"1,823","1,975.71",C,㈜건일토건,2013.07.03,2014.02.28,2014.03.01
2,경기150009,2015,0,24942,0,0,A,㈜경동건설,2012.09.03,2014.08.31,2012.10.31
3,부산110028,2012,2963,140,346,"6,547",A,경동건설(주),2007.01.19,2012.11.16,2011.09.06
4,부산110028,2011,2963,280,346.2,6,A,경동건설(주),2007.01.19,,2011.09.06


# Save pandas dataframe to csv ( nsdi companys )

In [6]:
# save nsdi_company ( wait very long time about 20min )
end_page = int(total_page())

company_df = pd.DataFrame(columns=["company", "local", "regno", "url"])
for page in range(1, end_page):
    company_df = page_crawling(page, company_df)
company_df.to_csv("./nsdi_company.csv", index=False)

# load nsdi_company
nsdi_company = pd.read_csv("./nsdi_company.csv")
nsdi_company.tail()

,company,local,regno,url
4805,휴앤산업개발㈜,41,경기086004,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4806,흥우건설 주식회사,26,부산090001,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4807,흥진종합건설(주),11,서울130026,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4808,흥한주택종합건설 주식회사,48,경남150027,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4809,희경건설㈜,11,서울080385,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...


# Save pandas dataframe to csv ( nsdi company detail )
- spend very long time about 40min for excute

In [7]:
# load nsdi_company 
company_df = pd.read_csv("./nsdi_company.csv")

# save nsdi_detail
detail_df = pd.DataFrame(columns=[
        "regno", 
        "year", 
        "business", 
        "benefit", 
        "land_width", 
        "build_width", 
        "nsdi_type",
        "company_name",
        "date1",
        "date2",
        "date3",        
    ])

for idx, row in company_df.iterrows():
    local = row["local"]
    regno = row["regno"]
    detail_df = company_crawling(local, regno, detail_df)
    
detail_df.to_csv("./nsdi_detail.csv", index=False)

# load nsdi_detail
nsdi_detail = pd.read_csv("./nsdi_detail.csv")
nsdi_detail = nsdi_detail.fillna("")
nsdi_detail.tail()

,regno,year,business,benefit,land_width,build_width,nsdi_type,company_name,date1,date2,date3
7540,경기086108,2015,0,0,0,0,A,효원산업개발(주),2013.01.05,2014.02.13,2014.03.01
7541,경기086108,2014,7800,840,"1,115.5","7,826.44",A,효원산업개발(주),2013.01.05,2014.02.13,2014.03.01
7542,경기086108,2011,2379,1130,"2,050","3,589",A,효원산업개발(주),2008.01.01,2008.06.11,
7543,경기086108,2010,2379,140,"2,050","3,589",A,효원산업개발,2008.01.01,2008.06.01,2010.03.05
7544,경기086108,2009,385,385,590,599,A,효원산업개발(주),2009.05.05,2009.08.14,2009.08.14


# Merge company data frame & detail data frame

In [8]:
company_df.tail()

,company,local,regno,url
4805,휴앤산업개발㈜,41,경기086004,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4806,흥우건설 주식회사,26,부산090001,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4807,흥진종합건설(주),11,서울130026,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4808,흥한주택종합건설 주식회사,48,경남150027,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...
4809,희경건설㈜,11,서울080385,http://nsdi.go.kr/?menuno=2775&devAct=view&sid...


In [9]:
nsdi_detail.tail()

,regno,year,business,benefit,land_width,build_width,nsdi_type,company_name,date1,date2,date3
7540,경기086108,2015,0,0,0,0,A,효원산업개발(주),2013.01.05,2014.02.13,2014.03.01
7541,경기086108,2014,7800,840,"1,115.5","7,826.44",A,효원산업개발(주),2013.01.05,2014.02.13,2014.03.01
7542,경기086108,2011,2379,1130,"2,050","3,589",A,효원산업개발(주),2008.01.01,2008.06.11,
7543,경기086108,2010,2379,140,"2,050","3,589",A,효원산업개발,2008.01.01,2008.06.01,2010.03.05
7544,경기086108,2009,385,385,590,599,A,효원산업개발(주),2009.05.05,2009.08.14,2009.08.14


In [10]:
# project data frame
summary_df = company_df.merge(nsdi_detail, on="regno")[["company","local","year","benefit"]]
summary_df.tail()

,company,local,year,benefit
7748,효원산업개발㈜,41,2015,0
7749,효원산업개발㈜,41,2014,840
7750,효원산업개발㈜,41,2011,1130
7751,효원산업개발㈜,41,2010,140
7752,효원산업개발㈜,41,2009,385


In [12]:
# benefit date frame of company 
company_benefit_df = summary_df.groupby("company").agg({"year":np.size,"benefit":[np.sum]})
company_benefit_df.fillna("").tail()

,benefit,year
,sum,size
company,,
화양시장(주),59000,1
화엄토건(주),19076,2
화이트코리아㈜,167357,4
효마을종합건설(주),0,2
효원산업개발㈜,2495,6


In [14]:
# benefit of company
year_benefit_df = summary_df.groupby(["company"]).agg({"benefit":[np.sum]})
year_benefit_df.tail()

,benefit
,sum
company,
화양시장(주),59000
화엄토건(주),19076
화이트코리아㈜,167357
효마을종합건설(주),0
효원산업개발㈜,2495
